This notebook is for who is new to using Gemma in Kaggle.  
Let's go through how to use it step by step.

**Acknowledgment**  
This notebook based on [LLM 20 Questions Starter Notebook](https://www.kaggle.com/code/ryanholbrook/llm-20-questions-starter-notebook)

Before we start, check the GPU setup.

<img width="400px" src=attachment:9319be98-0139-41f9-a741-6b606b7cadc6.png>

---

## Add Gemma model

1. Click `+ Add Input` button.
<img src=attachment:6dabe5ed-9e7e-437b-be73-ee0c3b3a2afe.png width="400">

2. Search `Gemma` model.
<img src=attachment:68d9d1fa-84f4-4c37-acf9-f5cb2b825fcb.png width="400">

3. Set parameters. (I used *2b-it* because *7b* is too slow to test fast)
<img src=attachment:c7f1744e-08df-4ba7-ab61-4b0cd4e4e6fd.png width="400">

4. You can see the model has added.
<img src=attachment:056f294d-eee3-4bbb-b68b-6bc991a9c746.png width="400">

In [ ]:
%%bash
# You can find input files in `/kaggle/input` directory.
tree /kaggle/input/gemma/pytorch

# Setup Gemma to code

To use Gemma model, you have to setup few things.

## Install dependencies

In [ ]:
%%bash
# Install python packages in `/kaggle/working/submission/lib` directory.
mkdir -p /kaggle/working/submission/lib
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece

# Download `gemma_pytorch` and move it to `/kaggle/working/submission/lib` directory.
cd /kaggle/working
git clone https://github.com/google/gemma_pytorch.git > /dev/null
mkdir /kaggle/working/submission/lib/gemma/
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/

In [ ]:
%%bash
# You can see the result.
tree /kaggle/working/submission/lib

In [ ]:
import sys

# Add module path to use libraries in `/kaggle/working/submission/lib` directory.
sys.path.insert(0, "/kaggle/working/submission/lib")

## Load Gemma model

In [ ]:
import torch
from gemma.config import get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM

# Set global parameters
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
VARIANT = "2b-it"
# VARIANT = "7b-it-quant"
WEIGHTS_PATH = f"/kaggle/input/gemma/pytorch/{VARIANT}/2" # input model path

print("current device is", DEVICE)
print("current Gemma model variant is", VARIANT)

In [ ]:
# Set model_config
def get_model_config(variant):
    if "2b" in variant:
        return get_config_for_2b()
    if "7b" in variant:
        return get_config_for_7b()
    raise ValueError(f"Not supported variant - {variant}")
model_config = get_model_config(VARIANT)
model_config.tokenizer = f"{WEIGHTS_PATH}/tokenizer.model"
model_config.quant = "quant" in VARIANT
torch.set_default_dtype(model_config.get_dtype())

# Load Gemma model
gemma_model = GemmaForCausalLM(model_config)
gemma_model.load_weights(f"{WEIGHTS_PATH}/gemma-{VARIANT}.ckpt")
gemma_model = gemma_model.to(DEVICE).eval()

## Use Gemma

### Simple QA

You can use the model with `generate` method.

In [ ]:
prompt = "Let me know how to use you."

model_answer = gemma_model.generate(
    prompts=prompt,
    device=torch.device(DEVICE),
    output_len=100, # max output tokens
    temperature=0.01,
    top_p=0.1,
    top_k=1,
)

print(model_answer)

### QA with instruction tuning

You can do the prompt engineering with instruction tuning.  
And you can see the code in detail in `GemmaFormatter` class of [Starter Notebook](https://www.kaggle.com/code/ryanholbrook/llm-20-questions-starter-notebook).

Reference: https://ai.google.dev/gemma/docs/formatting

In [ ]:
system_prompt = "You are an AI assistant designed to play the 20 Questions game. In this game, the Answerer thinks of a keyword and responds to yes-or-no questions by the Questioner. The keyword is a specific person, place, or thing."

prompt = "<start_of_turn>user\n" # GemmaFormatter.user
prompt += f"{system_prompt}<end_of_turn>\n"
prompt += "<start_of_turn>model\n" # GemmaFormatter.start_model_turn

print(prompt)

In [ ]:
model_answer = gemma_model.generate(
    prompts=prompt,
    device=torch.device(DEVICE),
    output_len=100, # max output tokens
    temperature=0.01,
    top_p=0.1,
    top_k=1,
)

print(model_answer)

---

I hope now you can use how to use the Gemma model for the competition  
and this notebook helps understand the starter notebook.